# Result figures for *Enhanced spatio-temporal electric load forecasts with less data using active deep learning*

---

## Overview
1. Import and test results
2. Numerical results
3. Space and time selection
4. Budget vs. accuracy
5. Training and validation losses against unqueried candidates
6. Validation losses against all candidates
7. Query sequence importance
8. Exemplar predictions
9. Manuscript: Results summary
10. Manuscript: Data selection maps

In this notebook session, we summarize and visualize the results of our experiments. First test if all numerical results were computed with the same hyper parameters. Next, we plot our results and exemplar predictions so as to visually see and evaluate their meaning. Lastly, we create figures for our manuscript from experiments that best visualize our findings. We start with importing a number of packages that we use throughout this notebook session. 

In [3]:
### Set some parameters ###

# choose which results dataset you want to process
profile_set = 'profiles_100'

# prediction types to consider
PRED_TYPE_LIST = [
    'spatial',
    'temporal',
    'spatio-temporal'
]

# parameter constellations to consider
PARAMETER_LIST = [
    'delta0_valup0', 
    'delta0_valup1', 
    'delta1_valup0', 
    'delta1_valup1'
]


# choose which AL variables to plot. Choose from 'X_t', 'X_s1', 'X_st', 'X_joint', 'X_(t,s)', 'Y_(t,s)'
AL_VARIABLES = [
#    'X_t', 
#    'X_s1', 
    'X_st', 
    'X_(t,s)', 
    'Y_hat_(t,s)', 
    'Y_(t,s)'
]


# choose which AL variables to plot. Choose from 'X_t', 'X_s1', 'X_st', 'X_joint', 'X_(t,s)', 'Y_(t,s)'
AL_VARIANTS = [
    'rnd d_c', 
    'max d_c', 
    'min d_c', 
    'avg d_c'
]


# create figure sub title list
fig_title_list = [
    'a.', 'b.', 'c.', 'd.', 'e.', 'f.', 'g.', 'h.',
    'i.', 'j.', 'k.', 'l.', 'm.', 'n.', 'o.', 'p.',
    'q.', 'r.', 's.', 't.', 'u.', 'v.', 'w.', 'x.'
]

# set width
WIDTH_FACTOR = 8

# set universal fontsize
FONTSIZE = 20

### Import packages ###

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d import Axes3D
import math
import tensorflow as tf
import numpy as np
from PIL import Image
import random


### Provide import paths and file names ###

# path to results folder
path_to_results = '../results/' + profile_set + '/'

# path to initially trained encoders and prediction model
path_to_encoders = path_to_results + 'encoder weights/'

# path to numeric result values
path_to_values = path_to_results + 'values/'

# path to unseen test samples
path_to_samples = path_to_results + 'samples/'

# path to models 
path_to_models = path_to_results + 'models/'

# individual file names
hyper_filename = 'hyper.csv'
results_filename = 'results.csv'
seqimportance_filename = 'sequence_importance.csv'
spacetime_selection_filename = 'spacetime_selection.csv'
budgetaccuracy_filename = 'budget_vs_accuracy.csv'

initial_model_filename = 'initial.h5'
PL_model_filename = 'PL.h5'

### Provide export paths and file names ###

# path to saving result figures
path_to_saving_figures = path_to_results + 'figures/'

if not os.path.exists(path_to_saving_figures):
    os.mkdir(path_to_saving_figures)

# path to subfolder for saving spacetime point selection
path_to_saving_spacetime = path_to_saving_figures + 'space-time selection/'

if not os.path.exists(path_to_saving_spacetime):
    os.mkdir(path_to_saving_spacetime)
    
# path to subfolder for saving budgetvsaccuracy
path_to_saving_budgetvsaccuracy = path_to_saving_figures + 'budget vs accuracy/'

if not os.path.exists(path_to_saving_budgetvsaccuracy):
    os.mkdir(path_to_saving_budgetvsaccuracy)
    
# path to subfolder for saving lossesvsunqueried
path_to_saving_lossesvsunqueried = path_to_saving_figures + 'training vs validation losses/'

if not os.path.exists(path_to_saving_lossesvsunqueried):
    os.mkdir(path_to_saving_lossesvsunqueried)
    
# path to subfolder for saving lossesvsall
path_to_saving_lossesvsall = path_to_saving_figures + 'validation against all/'

if not os.path.exists(path_to_saving_lossesvsall):
    os.mkdir(path_to_saving_lossesvsall)
    
# path to subfolder for saving seqimportance
path_to_saving_seqimportance = path_to_saving_figures + 'sequence importance/'

if not os.path.exists(path_to_saving_seqimportance):
    os.mkdir(path_to_saving_seqimportance)
    
# path to subfolder for saving exemplar predictions point selection
path_to_exemplar_predictions = path_to_saving_figures + 'exemplar predictions/'

if not os.path.exists(path_to_exemplar_predictions):
    os.mkdir(path_to_exemplar_predictions)
    
# path to subfolder for saving manuscript figure ADL data selection
path_to_ADL_selection = path_to_saving_figures + 'ADL selection/'

if not os.path.exists(path_to_ADL_selection):
    os.mkdir(path_to_ADL_selection)
    
# path to subfolder for saving manuscript figure ADL data selection
path_to_summary_results = path_to_saving_figures + 'summary results/'

if not os.path.exists(path_to_summary_results):
    os.mkdir(path_to_summary_results)

## 3. Space and time selection

Here, we visualize which points in time and space were queried during the experiments by our active learning and passive learning algorithms.

In [4]:
%%capture 
# prevents figures being printed out if used at begining of cell

# define some hyper for plots
n_iter_plot = 10
n_meshes_surface = 10

# import building meta data
path_to_building_meta = '../data/private/' + profile_set + '/meta/meta buildings.csv'
building_meta = pd.read_csv(path_to_building_meta)

def create_bottom_plot(first_colname, df_initial_sensors):
    
    ax.set_title(first_colname)

    # get bound coordinates of all buildings
    min_lat = df_initial_sensors['building lat'].min()
    max_lat = df_initial_sensors['building lat'].max() 
    min_long = df_initial_sensors['building long'].min()
    max_long = df_initial_sensors['building long'].max()

    # create evenly sized arrays and meshed grid of lats and longs
    lat_surface = np.linspace(
        min_lat, 
        max_lat,
        num=n_meshes_surface
    )
    long_surface = np.linspace(
        min_long, 
        max_long,
        num=n_meshes_surface
    )
    long_surface, lat_surface  = np.meshgrid(long_surface, lat_surface)
    
    map_height = 0
    Z = np.full((len(lat_surface), 1), map_height)
    
    ax.scatter(
        df_initial_sensors['building long'], 
        df_initial_sensors['building lat'], 
        map_height,  
        alpha=1, marker='x', c='r', s=100
    ) 
    ax.plot_surface(
        long_surface, 
        lat_surface, 
        Z,  
        alpha=0.03
    )
    ax.set_zlim(map_height)
    
def create_plot(time_data, df, df_new_sensors):
    
    # get bound coordinates of all buildings
    min_lat = df['building lat'].min()
    max_lat = df['building lat'].max() 
    min_long = df['building long'].min()
    max_long = df['building long'].max()

    # create evenly sized arrays and meshed grid of lats and longs
    lat_surface = np.linspace(
        min_lat, 
        max_lat,
        num=n_meshes_surface
    )
    long_surface = np.linspace(
        min_long, 
        max_long,
        num=n_meshes_surface
    )
    long_surface, lat_surface  = np.meshgrid(long_surface, lat_surface)
    
    # update max time point
    min_time_point, max_time_point = min(time_data), max(time_data)
    map_height = max_time_point/3 * (1 + 3* iteration/n_iter_plot)
    shifting_time = max_time_point - map_height
    Z = np.full((len(lat_surface), 1), map_height)
    
    ax.scatter(
        df['building long'], 
        df['building lat'], 
        time_data - shifting_time, 
        c=time_data, alpha=0.7
    )
    ax.scatter(
        df_new_sensors['building long'], 
        df_new_sensors['building lat'], 
        map_height,  
        alpha=1, marker='x', c='r', s=100
    )
    ax.plot_surface(
        long_surface, 
        lat_surface, 
        Z,  
        alpha=0.03
    )
    ax.set_zlim(min_time_point - shifting_time, max_time_point)
    
    
def customize_plot(iteration=None):
    
    # set angle
    ax.view_init(30, 103)
    
    # Get rid of the panes
    ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))

    # Get rid of the ticks
    ax.set_xticks([]) 
    ax.set_yticks([]) 
    ax.set_zticks([])

    # Add the labels
    ax.set_xlabel('longitude' )
    ax.set_ylabel('latitude')
    ax.set_zlabel('time')

    # shift time (z) axis
    tmp_planes = ax.zaxis._PLANES 
    ax.zaxis._PLANES = ( 
        tmp_planes[2], tmp_planes[3], 
        tmp_planes[0], tmp_planes[1], 
        tmp_planes[4], tmp_planes[5]
    )
    
    # shift lat (y) axis
    ax.yaxis._PLANES = ( 
        tmp_planes[2], tmp_planes[3], 
        tmp_planes[0], tmp_planes[1], 
        tmp_planes[4], tmp_planes[5]
    )
    
    
    # set subplot titles
    if iteration is not None:
        ax.set_title('iteration {}'.format(iteration+1))


# create a counter over the list of numeric results
result_index_counter = 0

# iterate over all considered prediction types
for index_pred, pred_type in enumerate(PRED_TYPE_LIST):
    
    # iterate over all considered parameter constellations
    for index_param, parameter in enumerate(PARAMETER_LIST):
        
        # get results df corresponding to currently iterated parameter and pred_type
        spacetime_result = spacetime_selection_list[result_index_counter]

        # increment result index counter
        result_index_counter += 1
        
        for AL_variable in AL_VARIABLES:
            
            for AL_variant in AL_VARIANTS:
        
                # create figure
                fig = plt.figure(figsize=(16, (n_iter_plot+1) * 8))

                # set the fontsize for figures
                mpl.rcParams.update({'font.size': 16}) #FONTSIZE

                colname_initial_sensors = '{} - initial sensors'.format(pred_type)
                intial_sensors = spacetime_result[colname_initial_sensors]
                df_initial_sensors = pd.DataFrame({'building ID':intial_sensors})
                df_initial_sensors = df_initial_sensors.merge(building_meta, on='building ID', how='left')
                
                plot_counter= 1
                ax = fig.add_subplot((n_iter_plot+1), 2, plot_counter, projection='3d')
                create_bottom_plot(
                    'Active deep learning (ADL) \n iteration 0',
                    df_initial_sensors
                )
                customize_plot()
                plot_counter += 1
                ax = fig.add_subplot((n_iter_plot+1), 2, plot_counter, projection='3d')
                create_bottom_plot(
                    'Passive deep learning (PDL) \n iteration 0',
                    df_initial_sensors
                )
                customize_plot()
                plot_counter += 1
                old_senors_AL_set = set(intial_sensors)
                old_senors_PL_set = set(intial_sensors)
                for iteration in range(n_iter_plot):

                    # create column names
                    colname_AL_times = '{} {} {} - iter {} time'.format(pred_type, AL_variable, AL_variant, iteration) 
                    colname_AL_spaces = '{} {} {} - iter {} space'.format(pred_type, AL_variable, AL_variant, iteration) 
                    colname_PL_times = '{} None PL - iter {} time'.format(pred_type, iteration)
                    colname_PL_spaces = '{} None PL - iter {} space'.format(pred_type, iteration)
                    
                    ### Plot AL results on left column ###
                    
                    # get data
                    space_data_AL = spacetime_result[colname_AL_spaces]
                    time_data_AL = spacetime_result[colname_AL_times]
                    space_data_PL = spacetime_result[colname_PL_spaces]
                    time_data_PL = spacetime_result[colname_PL_times]

                    # get new sensors and set old sensors
                    new_sensors_AL_set = set(space_data_AL).union(old_senors_AL_set)
                    new_sensors_PL_set = set(space_data_PL).union(old_senors_PL_set)
                    new_sensors_AL_list = list(new_sensors_AL_set - old_senors_AL_set)
                    new_sensors_PL_list = list(new_sensors_PL_set - old_senors_PL_set)
                    old_senors_AL_set = new_sensors_AL_set
                    old_senors_PL_set = new_sensors_PL_set
                    
                    # assign lat and long to building IDs
                    df_AL = pd.DataFrame({'building ID':space_data_AL})
                    df_AL = df_AL.merge(building_meta, on='building ID', how='left')
                    df_PL = pd.DataFrame({'building ID':space_data_PL})
                    df_PL = df_PL.merge(building_meta, on='building ID', how='left')

                    df_new_sensors_AL = pd.DataFrame({'building ID':new_sensors_AL_list})
                    df_new_sensors_AL = df_new_sensors_AL.merge(building_meta, on='building ID', how='left')
                    df_new_sensors_PL = pd.DataFrame({'building ID':new_sensors_PL_list})
                    df_new_sensors_PL = df_new_sensors_PL.merge(building_meta, on='building ID', how='left')

                    # AL temporal scatter plot
                    ax = fig.add_subplot((n_iter_plot+1), 2, plot_counter, projection='3d')
                    create_plot(time_data_AL, df_AL, df_new_sensors_AL)
                    customize_plot(iteration)
                    plot_counter += 1

                    # PL temporal scatter plot
                    ax = fig.add_subplot((n_iter_plot+1), 2, plot_counter, projection='3d')
                    create_plot(time_data_PL, df_PL, df_new_sensors_PL)
                    customize_plot(iteration)
                    plot_counter+= 1


                # create saving paths 
                saving_path = (
                    path_to_saving_spacetime 
                    + pred_type
                    + ' '
                    + parameter
                    + ' '
                    + AL_variable
                    + ' '
                    + AL_variant
                    + '.pdf'
                )

                # create a legend
                legend_elements = [
                    Line2D([0], [0], marker='o', color='w', markerfacecolor='b', markersize=10, label='query in space-time'),
                    Line2D([0], [0], marker='X', color='w', markerfacecolor='r', markersize=15, label='new sensor in space')
                ]

                # set layout tight
                fig.tight_layout()

                fig.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5,0.99))

                # save figures
                fig.savefig(saving_path, bbox_inches="tight")

## 4. Budget vs. accuracy

In [5]:
%%capture 
# prevents figures being printed out if used at begining of cell

for AL_variable in AL_VARIABLES:
    for AL_variant in AL_VARIANTS:
        
        # set the fontsize for figures
        mpl.rcParams.update({'font.size': FONTSIZE})

        fig, ax = plt.subplots(
            len(PRED_TYPE_LIST), 
            2, 
            figsize=(
                20, 
                8 * len(PRED_TYPE_LIST)
            )
        )

        # set plot_counter for subplot title setting
        plot_counter = 0

        # create a counter over the list of numeric results
        result_index_counter = 0

        # iterate over all considered prediction types
        for index_pred, pred_type in enumerate(PRED_TYPE_LIST):

            # iterate over all considered parameter constellations
            for index_param, parameter in enumerate(PARAMETER_LIST):

                delta = int(parameter[5])
                valup = int(parameter[12])

                # skip cases where we validate against queried data too
                if valup == 0:
                    # increment result index counter
                    result_index_counter += 1
                    continue

                # get results df corresponding to currently iterated parameter and pred_type
                budgetaccuracy = budgetaccuracy_list[result_index_counter]

                # increment result index counter
                result_index_counter += 1

                # create the column name for PL lossess
                col_name_data = (
                    pred_type 
                    + ' None ' 
                    + 'PL ' 
                    + 'data'
                )
                col_name_sensors = (
                    pred_type 
                    + ' None ' 
                    + 'PL ' 
                    + 'sensors'
                )
                col_name_streamtimes = (
                    pred_type 
                    + ' None ' 
                    + 'PL ' 
                    + 'streamtimes'
                )
                col_name_accuracy = (
                    pred_type 
                    + ' None ' 
                    + 'PL ' 
                    + 'accuracy'
                )

                # get PL results
                PL_data = np.append(0, budgetaccuracy[col_name_data].values)
                PL_accuracy = np.append(0, budgetaccuracy[col_name_accuracy].values)

                # create the column name for iterated validation loss
                col_name_data = (
                    pred_type 
                    + ' ' 
                    + AL_variable 
                    + ' ' 
                    + AL_variant 
                    + ' data'
                )
                col_name_accuracy = (
                    pred_type 
                    + ' ' 
                    + AL_variable 
                    + ' ' 
                    + AL_variant 
                    + ' accuracy'
                )

                # get training losses for mode 1 with validation updates
                AL_data = np.append(0, budgetaccuracy[col_name_data].values)
                AL_accuracy = np.append(0, budgetaccuracy[col_name_accuracy].values)

                if len(PRED_TYPE_LIST) == 1:
                    plot_ax = ax[delta]
                else:
                    plot_ax = ax[index_pred, delta]
                    
                    
                # plot iterated training losses
                plot_ax.plot(
                    AL_accuracy,
                    color='b'
                )

                for x,y in enumerate(AL_accuracy):

                    # plot annotations only on every second step
                    if (x+1)%2 == 0:
                        plot_ax.annotate(
                            str(AL_data[x])+'%',
                            (x, y+5)
                        )

                # plot PL accuracy.
                # Note: Moved plotting down after plotting AL, in order to have legends aligned with height of plots
                plot_ax.plot(
                    PL_accuracy, 
                    color='r',
                )

                plot_ax.set_ylim(
                    top=100
                )

                for x,y in enumerate(PL_accuracy):
                    # plot annotations only on every second step
                    if (x+1)%2 == 0:
                        plot_ax.annotate(
                            str(PL_data[x])+'%',
                            (x, y-5)
                        )

                
                # set subplot titles
                plot_ax.set_title(fig_title_list[plot_counter])
                plot_counter+= 1
                
                if len(PRED_TYPE_LIST) == 1:
                    # set y-axis labels
                    ax[0].set_ylabel(
                        '{} \n prediction accuracy'.format(pred_type), 
                        fontsize=FONTSIZE
                    )

                    # set x-axis
                    ax[delta].set_xlabel(
                        'data selection \n iteration', 
                        fontsize=FONTSIZE
                    )
                else:
                    # set y-axis labels
                    ax[index_pred, 0].set_ylabel(
                        '{} \n prediction accuracy'.format(pred_type), 
                        fontsize=FONTSIZE
                    )

                    # set x-axis
                    ax[len(PRED_TYPE_LIST)-1, delta].set_xlabel(
                        'data selection \n iteration', 
                        fontsize=FONTSIZE
                    )


        # set column titles
        cols = [
            'δ=0 \n a.', 
            'δ=1 \n b.'
        ]
        if len(PRED_TYPE_LIST) == 1:
            for axes, col in zip(ax, cols):
                axes.set_title(col)
        else:
            for axes, col in zip(ax[0], cols):
                axes.set_title(col)

        # create saving paths 
        saving_path = (
            path_to_saving_budgetvsaccuracy 
            + AL_variable
            + ' '
            + AL_variant
            + '.pdf'
        )

        legend_elements = [Line2D([0], [0], color='b', label='Active learning', markersize=FONTSIZE),
                           Line2D([0], [0], color='r', label='Passive learning', markersize=FONTSIZE),
                           Line2D([0], [0], color='w', label='% = budget usage', markersize=FONTSIZE)]

        # set layout tight
        fig.tight_layout()

        fig.legend(
            handles=legend_elements, 
            loc='upper center', 
            bbox_to_anchor=(0.9,1.08 - 0.02 *len(PRED_TYPE_LIST))
        )

        # save figures
        fig.savefig(saving_path, bbox_inches="tight")

## 8. Exemplar predictions

For each experiment, we have saved a sample of 1,000 randomly chosen data points that were not queried by our passive learning or active learning algorithm when these terminated. Here, we loadfive exemplar points for each of our conducted experiments from these at random and compare the prediction of each of our three models. The first column represents the performance of the initial model prior to being further trained on any data from the candidate data pool. The second column represents the performance of our passively trained benchmark model. The third column represents the performance of our actively trained prediction model.

What we ideally want to observe visually is that the prediction performance of our actively trained models is approximately as good as the performance of our passively trained models, as we are able to use fewer data points and sensors with our active learning models. 

In [9]:
%%capture 
# prevents figures being printed out if used at begining of cell

# choose how many random datapoints to predict and plot
N_DATAPOINTS = 5

# number of columns to plot data points. Keep this 3
n_cols = 3

# choose a different font size for these figures
FONTSIZE = 14

# set the fontsize
mpl.rcParams.update({'font.size': FONTSIZE})

# iterate over all considered prediction types
for index_pred, pred_type in enumerate(PRED_TYPE_LIST):
    
    # iterate over all considered parameter constellations
    for index_param, parameter in enumerate(PARAMETER_LIST):
  
        delta = int(parameter[5])
        valup = int(parameter[12]) 

        # provide paths to initial and PL models and samples
        path_to_initial_model = (
            path_to_models 
            + parameter 
            + '/' 
            + pred_type 
            + '/' 
            + initial_model_filename
        )
        path_to_PL_model = (
            path_to_models 
            + parameter 
            + '/' 
            + pred_type 
            + '/' 
            + PL_model_filename
        )
        path_to_PL_data = (
            path_to_samples 
            + parameter 
            + '/' 
            + pred_type
            + '/' 
            + 'PL_'
        )
        
        # import models and samples for PL
        initial_model =  tf.keras.models.load_model(
            path_to_initial_model, 
            compile=False
        )
        PL_model = tf.keras.models.load_model(
            path_to_PL_model, 
            compile=False
        )
        
        path_to_file = path_to_PL_data + 'X_t.npy' 
        X_t = np.load(path_to_file)

        path_to_file = path_to_PL_data + 'X_s.npy' 
        X_s = np.load(path_to_file)

        path_to_file = path_to_PL_data + 'X_s1.npy' 
        X_s1 = np.load(path_to_file)

        path_to_file = path_to_PL_data + 'X_st.npy' 
        X_st = np.load(path_to_file)

        path_to_file = path_to_PL_data + 'Y.npy' 
        Y_pl = np.load(path_to_file)
        
        # make predictions
        initial_predictions = initial_model.predict(
            [X_t, X_s1, X_st]
        )
        PL_predictions = PL_model.predict(
            [X_t, X_s1, X_st]
        )

        # iterate over all AL variables
        for index_var, AL_variable in enumerate(AL_VARIABLES):
            
            # iterate over all AL variants
            for index_method, AL_variant in enumerate(AL_VARIANTS):
                
                # create figure
                fig, ax = plt.subplots(
                    N_DATAPOINTS, 
                    n_cols, 
                    sharex=True , 
                    figsize=(16, N_DATAPOINTS * 4)
                )

                AL_model_filename = '{} {}.h5'.format(
                    AL_variable, 
                    AL_variant
                )
                AL_sample_name = '{} {} '.format(
                    AL_variable, 
                    AL_variant
                )
                
                # provide paths to initial and PL models and samples
                path_to_AL_model = (
                    path_to_models 
                    + parameter 
                    + '/' 
                    + pred_type 
                    + '/' 
                    + AL_model_filename
                )
                path_to_AL_data = (
                    path_to_samples 
                    + parameter 
                    + '/' 
                    + pred_type 
                    + '/' 
                    + AL_sample_name
                )
                
                
                # import models and samples for AL
                AL_model =  tf.keras.models.load_model(
                    path_to_AL_model, 
                    compile=False
                )
        
                path_to_file = path_to_AL_data + 'X_t.npy' 
                X_t = np.load(path_to_file)

                path_to_file = path_to_AL_data + 'X_s.npy' 
                X_s = np.load(path_to_file)

                path_to_file = path_to_AL_data + 'X_s1.npy' 
                X_s1 = np.load(path_to_file)

                path_to_file = path_to_AL_data + 'X_st.npy' 
                X_st = np.load(path_to_file)

                path_to_file = path_to_AL_data + 'Y.npy' 
                Y_al = np.load(path_to_file)
                
                # make predictions
                AL_predictions = AL_model.predict(
                    [X_t, X_s1, X_st]
                )
                
                # plot predictions for randomly chosen points
                rnd_index_array_initial = np.random.choice(
                    np.arange(len(Y_pl)),
                    N_DATAPOINTS
                )
                rnd_index_array_PL = np.random.choice(
                    np.arange(len(Y_pl)),
                    N_DATAPOINTS
                )
                rnd_index_array_AL = np.random.choice(
                    np.arange(len(Y_al)), 
                    N_DATAPOINTS
                )

                title_counter = 0
                
                # iterate over each row of figure
                for row in range(N_DATAPOINTS):
                    
                    plot1 = ax[row, 0].plot(
                        initial_predictions[
                            rnd_index_array_initial[
                                row
                            ]
                        ]
                    )
                    ax[row, 1].plot(
                        PL_predictions[
                            rnd_index_array_PL[
                                row
                            ]
                        ]
                    )
                    ax[row, 2].plot(
                        AL_predictions[
                            rnd_index_array_AL[
                                row
                            ]
                        ]
                    )
                    
                    plot2 = ax[row, 0].plot(
                        Y_pl[
                            rnd_index_array_initial[
                                row
                            ]
                        ]
                    )
                    ax[row, 1].plot(
                        Y_pl[
                            rnd_index_array_PL[
                                row
                            ]
                        ]
                    )
                    ax[row, 2].plot(
                        Y_al[
                            rnd_index_array_AL[
                                row
                            ]
                        ]
                    )
                    
                    # set title
                    for col in range(n_cols):
                        ax[row, col].set_title(fig_title_list[title_counter])
                        title_counter +=1
                
                # add a figure legend
                fig.legend(
                    [plot1, plot2], 
                    labels=['true load profile', 'predicted load profile'], 
                    loc='upper center', 
                    bbox_to_anchor=(0.8, 0.95)
                )

                colname_list = [
                    'Initial model \n a.', 
                    'Passive learning \n b.', 
                    'Active learning \n c.'
                ]

                # set col names
                for axes, colname in zip(ax[0], colname_list):
                    axes.set_title(colname)

                # set one y- and x-axis for all sub plots
                fig.add_subplot(111, frame_on=False)
                plt.tick_params(
                    labelcolor="none", 
                    bottom=False, 
                    left=False
                )
                plt.xlabel(
                    'time [15-min]', 
                    fontsize=FONTSIZE+3
                )
                plt.ylabel(
                    'building electric consumption [kW]', 
                    fontsize=FONTSIZE+3
                )
                
                filename = '{} {} {} delta{} valup{}.pdf'.format(
                    pred_type, 
                    AL_variable, 
                    AL_variant, 
                    delta, 
                    valup
                )

                saving_path = (
                    path_to_exemplar_predictions 
                    + filename
                )

                fig.savefig(saving_path)

## 10. Manuscript: Data selection maps

Our findings can have important implications for the energy transition to a carbon free power systems and therefore for mitigating climate change. The ADL method we propose can be used by distribution and transmission system operators for electricity to make overall more accurate predictions of electric load using budgets for installing smart meters and streaming their data more effectively. This figure is supposed to visualize how ADL can show us where to place sensors and when to collect their data as we proceed in time (a. - f.). We start with choosing a set of locations to place sensors uniformly at random (a.). Then, we collect training data for making spatio-temporal predictions and learn where to best place a relatively large set of new sensors next (b.). Next, we iteratively place fewer sensors to make better predictions in the most informative sequence (c. - f.).

In [11]:
%%capture 
# prevents figures being printed out if used at begining of cell


# define some hyper for plots
n_rows = 3
n_cols = 2
n_iter_plot = n_rows * n_cols - 1
n_meshes_surface = 10
FONTSIZE = 22

# import building meta data
path_to_building_meta = '../data/private/' + profile_set + '/meta/meta buildings.csv'
building_meta = pd.read_csv(path_to_building_meta)

def create_bottom_plot(df_initial_sensors):

    # get bound coordinates of all buildings
    min_lat = df_initial_sensors['building lat'].min()
    max_lat = df_initial_sensors['building lat'].max() 
    min_long = df_initial_sensors['building long'].min()
    max_long = df_initial_sensors['building long'].max()

    # create evenly sized arrays and meshed grid of lats and longs
    lat_surface = np.linspace(
        min_lat, 
        max_lat,
        num=n_meshes_surface
    )
    long_surface = np.linspace(
        min_long, 
        max_long,
        num=n_meshes_surface
    )
    long_surface, lat_surface  = np.meshgrid(long_surface, lat_surface)
    
    map_height = 0
    Z = np.full((len(lat_surface), 1), map_height)
    
    ax.scatter(
        df_initial_sensors['building long'], 
        df_initial_sensors['building lat'], 
        map_height,  
        alpha=1, marker='x', c='r', s=100
    ) 
    ax.plot_surface(
        long_surface, 
        lat_surface, 
        Z,  
        alpha=0.03
    )
    ax.set_zlim(map_height)
    
def create_plot(time_data, df, df_new_sensors):
    
    # get bound coordinates of all buildings
    min_lat = df['building lat'].min()
    max_lat = df['building lat'].max() 
    min_long = df['building long'].min()
    max_long = df['building long'].max()

    # create evenly sized arrays and meshed grid of lats and longs
    lat_surface = np.linspace(
        min_lat, 
        max_lat,
        num=n_meshes_surface
    )
    long_surface = np.linspace(
        min_long, 
        max_long,
        num=n_meshes_surface
    )
    long_surface, lat_surface  = np.meshgrid(long_surface, lat_surface)
    
    # update max time point
    min_time_point, max_time_point = min(time_data), max(time_data)
    map_height = max_time_point/3 * (1 + 3* iteration/n_iter_plot)
    shifting_time = max_time_point - map_height
    Z = np.full((len(lat_surface), 1), map_height)
    
    ax.scatter(
        df['building long'], 
        df['building lat'], 
        time_data - shifting_time, 
        c=time_data, alpha=0.7
    )
    ax.scatter(
        df_new_sensors['building long'], 
        df_new_sensors['building lat'], 
        map_height,  
        alpha=1, marker='x', c='r', s=100
    )
    ax.plot_surface(
        long_surface, 
        lat_surface, 
        Z,  
        alpha=0.03
    )
    ax.set_zlim(min_time_point - shifting_time, max_time_point)
    

def customize_plot(iteration):
    
    # set angle
    ax.view_init(30, 110)
    
    # Get rid of the panes
    ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))

    # Get rid of the ticks
    ax.set_xticks([]) 
    ax.set_yticks([]) 
    ax.set_zticks([])

    # Add the labels
    ax.set_xlabel('long', fontsize=FONTSIZE)
    ax.set_ylabel('lat', fontsize=FONTSIZE)
    ax.set_zlabel('time', fontsize=FONTSIZE)

    # shift time (z) axis
    tmp_planes = ax.zaxis._PLANES 
    ax.zaxis._PLANES = ( 
        tmp_planes[2], tmp_planes[3], 
        tmp_planes[0], tmp_planes[1], 
        tmp_planes[4], tmp_planes[5]
    )
    
    # shift lat (y) axis
    ax.yaxis._PLANES = ( 
        tmp_planes[2], tmp_planes[3], 
        tmp_planes[0], tmp_planes[1], 
        tmp_planes[4], tmp_planes[5]
    )
    
    
    # set subplot titles
    ax.set_title(
        fig_title_list[plot_counter-1].format(iteration+1), 
        fontsize=FONTSIZE + 4
    )


# create a counter over the list of numeric results
result_index_counter = 0

# iterate over all considered prediction types
for index_pred, pred_type in enumerate(PRED_TYPE_LIST):
    
    # iterate over all considered parameter constellations
    for index_param, parameter in enumerate(PARAMETER_LIST):
        
        # get results df corresponding to currently iterated parameter and pred_type
        spacetime_result = spacetime_selection_list[result_index_counter]

        # increment result index counter
        result_index_counter += 1
        
        for AL_variable in AL_VARIABLES:
            
            for AL_variant in AL_VARIANTS:
        
                # create figure
                fig = plt.figure(figsize=(16, n_rows * 8))

                # set the fontsize for figures
                mpl.rcParams.update({'font.size': 16}) #FONTSIZE

                colname_initial_sensors = '{} - initial sensors'.format(pred_type)
                intial_sensors = spacetime_result[colname_initial_sensors]
                df_initial_sensors = pd.DataFrame({'building ID':intial_sensors})
                df_initial_sensors = df_initial_sensors.merge(building_meta, on='building ID', how='left')
                
                plot_counter= 1
                ax = fig.add_subplot(n_rows, n_cols, plot_counter, projection='3d')
                create_bottom_plot(df_initial_sensors)
                customize_plot(iteration=0)
                plot_counter += 1
                old_senors_AL_set = set(intial_sensors)
                old_senors_PL_set = set(intial_sensors)
                for iteration in range(n_iter_plot):

                    # create column names
                    colname_AL_times = '{} {} {} - iter {} time'.format(pred_type, AL_variable, AL_variant, iteration) 
                    colname_AL_spaces = '{} {} {} - iter {} space'.format(pred_type, AL_variable, AL_variant, iteration) 
                    
                    ### Plot AL results on left column ###
                    
                    # get data
                    space_data_AL = spacetime_result[colname_AL_spaces]
                    time_data_AL = spacetime_result[colname_AL_times]

                    # get new sensors and set old sensors
                    new_sensors_AL_set = set(space_data_AL).union(old_senors_AL_set)
                    new_sensors_AL_list = list(new_sensors_AL_set - old_senors_AL_set)
                    old_senors_AL_set = new_sensors_AL_set
                    
                    # assign lat and long to building IDs
                    df_AL = pd.DataFrame({'building ID':space_data_AL})
                    df_AL = df_AL.merge(building_meta, on='building ID', how='left')

                    df_new_sensors_AL = pd.DataFrame({'building ID':new_sensors_AL_list})
                    df_new_sensors_AL = df_new_sensors_AL.merge(building_meta, on='building ID', how='left')

                    # AL temporal scatter plot
                    ax = fig.add_subplot(n_rows, n_cols, plot_counter, projection='3d')
                    create_plot(time_data_AL, df_AL, df_new_sensors_AL)
                    customize_plot(iteration)
                    plot_counter += 1

                # create saving paths 
                saving_path = (
                    path_to_ADL_selection 
                    + pred_type
                    + ' '
                    + parameter
                    + ' '
                    + AL_variable
                    + ' '
                    + AL_variant
                    + '.pdf'
                )

                # create a legend
                legend_elements = [
                    Line2D([0], [0], marker='o', color='w', markerfacecolor='b', markersize=10, label='query in space-time'),
                    Line2D([0], [0], marker='X', color='w', markerfacecolor='r', markersize=15, label='new sensor in space')
                ]

                # set layout tight
                fig.tight_layout()

                fig.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(0.6, 1), fontsize=FONTSIZE)

                # save figures
                fig.savefig(saving_path, bbox_inches="tight")